# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
col_names = ['target', 'id', 'date', 'flag', 'user', 'text']

In [3]:
# your code here

df = pd.read_csv('./data/training.1600000.processed.noemoticon.csv',encoding='latin1', header=None, names=col_names)

In [4]:
df_reduced = df.sample(20000)
df_reduced

,target,id,date,flag,user,text
1054413,4,1961988118,Fri May 29 10:20:58 PDT 2009,NO_QUERY,mariaam1004,"1, 300 updates!!!!"
134725,0,1836333909,Mon May 18 07:48:39 PDT 2009,NO_QUERY,zeisch,@afimomogum RM15! but i vont be at homegrown
1010905,4,1881047963,Fri May 22 03:25:17 PDT 2009,NO_QUERY,Phylomene,@itslauraaa AHAH WHat is it pie???
1410911,4,2056371511,Sat Jun 06 11:01:26 PDT 2009,NO_QUERY,dianebr25,i just wached shanedawson's new vid it frekin ...
1520429,4,2176197871,Mon Jun 15 03:11:42 PDT 2009,NO_QUERY,wardrox,"@yarpp all seems to work. Ah well, I guess my ..."
...,...,...,...,...,...,...
1471485,4,2065215882,Sun Jun 07 08:08:27 PDT 2009,NO_QUERY,spulian,oooh catch me if you can is on
1219796,4,1989982507,Mon Jun 01 03:29:13 PDT 2009,NO_QUERY,Sabbelusa,watched &quot;the Junglebook&quot; w/ friends ...
556824,0,2204373161,Wed Jun 17 02:02:43 PDT 2009,NO_QUERY,alcoholiccheese,They do it cause they hate me http://twitpic....
159664,0,1956889553,Thu May 28 22:57:16 PDT 2009,NO_QUERY,scottfotki,@MarieLuv dammit...... try again?


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# your code here

def clean_up(s):
    
    url = re.sub('http://[\\w]+\\.\\w+', '', s)
    numbers = re.sub('\\d+', '', url).lower()
    new_string = re.sub('[^a-z0-9]', ' ', numbers).strip()
    
    return new_string

In [6]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/usuario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def tokenize(s):
    
    return word_tokenize(s)

In [8]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()

In [9]:
def stem_and_lemmatize(l):

    return [porter.stem(i) for i in l]

In [10]:
from nltk.corpus import stopwords

In [11]:
def remove_stopwords(l):

        stop_words = set(stopwords.words('english'))
        
        return [w for w in l if w not in stop_words]

In [12]:
df_reduced['text_processed'] = df_reduced['text'].apply(clean_up)

In [13]:
df_reduced['text_processed'] = df_reduced['text_processed'].apply(tokenize, stem_and_lemmatize)

In [14]:
df_reduced['text_processed'] = df_reduced['text_processed'].apply(remove_stopwords)

In [15]:
df_reduced

,target,id,date,flag,user,text,text_processed
1054413,4,1961988118,Fri May 29 10:20:58 PDT 2009,NO_QUERY,mariaam1004,"1, 300 updates!!!!",[updates]
134725,0,1836333909,Mon May 18 07:48:39 PDT 2009,NO_QUERY,zeisch,@afimomogum RM15! but i vont be at homegrown,"[afimomogum, rm, vont, homegrown]"
1010905,4,1881047963,Fri May 22 03:25:17 PDT 2009,NO_QUERY,Phylomene,@itslauraaa AHAH WHat is it pie???,"[itslauraaa, ahah, pie]"
1410911,4,2056371511,Sat Jun 06 11:01:26 PDT 2009,NO_QUERY,dianebr25,i just wached shanedawson's new vid it frekin ...,"[wached, shanedawson, new, vid, frekin, rocked]"
1520429,4,2176197871,Mon Jun 15 03:11:42 PDT 2009,NO_QUERY,wardrox,"@yarpp all seems to work. Ah well, I guess my ...","[yarpp, seems, work, ah, well, guess, issue, e..."
...,...,...,...,...,...,...,...
1471485,4,2065215882,Sun Jun 07 08:08:27 PDT 2009,NO_QUERY,spulian,oooh catch me if you can is on,"[oooh, catch]"
1219796,4,1989982507,Mon Jun 01 03:29:13 PDT 2009,NO_QUERY,Sabbelusa,watched &quot;the Junglebook&quot; w/ friends ...,"[watched, quot, junglebook, quot, w, friends, ..."
556824,0,2204373161,Wed Jun 17 02:02:43 PDT 2009,NO_QUERY,alcoholiccheese,They do it cause they hate me http://twitpic....,"[cause, hate, lybd]"
159664,0,1956889553,Thu May 28 22:57:16 PDT 2009,NO_QUERY,scottfotki,@MarieLuv dammit...... try again?,"[marieluv, dammit, try]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [16]:
# your code here
from nltk.probability import FreqDist

In [17]:
# your code here
cfdist = FreqDist()

def freq_dist(x):
    for i in x:
        cfdist[i] += 1

In [19]:
df_reduced['text_processed'].apply(lambda x: freq_dist(x))

1054413    None
134725     None
1010905    None
1410911    None
1520429    None
           ... 
1471485    None
1219796    None
556824     None
159664     None
325398     None
Name: text_processed, Length: 20000, dtype: object

In [20]:
common_words = cfdist.most_common(5000)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [21]:
df_reduced['positive'] = df['target'].apply(lambda x: False if x == 0 else True)

In [22]:
df_reduced.head()

,target,id,date,flag,user,text,text_processed,positive
1054413,4,1961988118,Fri May 29 10:20:58 PDT 2009,NO_QUERY,mariaam1004,"1, 300 updates!!!!",[updates],True
134725,0,1836333909,Mon May 18 07:48:39 PDT 2009,NO_QUERY,zeisch,@afimomogum RM15! but i vont be at homegrown,"[afimomogum, rm, vont, homegrown]",False
1010905,4,1881047963,Fri May 22 03:25:17 PDT 2009,NO_QUERY,Phylomene,@itslauraaa AHAH WHat is it pie???,"[itslauraaa, ahah, pie]",True
1410911,4,2056371511,Sat Jun 06 11:01:26 PDT 2009,NO_QUERY,dianebr25,i just wached shanedawson's new vid it frekin ...,"[wached, shanedawson, new, vid, frekin, rocked]",True
1520429,4,2176197871,Mon Jun 15 03:11:42 PDT 2009,NO_QUERY,wardrox,"@yarpp all seems to work. Ah well, I guess my ...","[yarpp, seems, work, ah, well, guess, issue, e...",True


In [25]:
columns = common_words

In [27]:
bag_words = []

for index,row in df_reduced.iterrows():
    processed_words = {}
    for i in columns:
         processed_words[i] = i in row['text_processed']

    bag_words.append((processed_words, row['positive']))

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [30]:
# your code here
import nltk

train_set, test_set = bag_words[:500], bag_words[:500]

classifier = nltk.NaiveBayesClassifier.train(train_set)

classifier.show_most_informative_features()

Most Informative Features
             ('aaah', 4) = False           False : True   =      1.0 : 1.0
              ('aah', 5) = False           False : True   =      1.0 : 1.0
             ('aane', 3) = False           False : True   =      1.0 : 1.0
            ('aaron', 7) = False           False : True   =      1.0 : 1.0
             ('abby', 4) = False           False : True   =      1.0 : 1.0
              ('abc', 4) = False           False : True   =      1.0 : 1.0
          ('ability', 3) = False           False : True   =      1.0 : 1.0
             ('abit', 5) = False           False : True   =      1.0 : 1.0
            ('able', 54) = False           False : True   =      1.0 : 1.0
              ('abs', 3) = False           False : True   =      1.0 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [31]:
# your code here

nltk.classify.accuracy(classifier, test_set)

0.514

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here